In [10]:
# Softmax example in TF using the classical Iris dataset
# Download iris.data from https://archive.ics.uci.edu/ml/datasets/Iris

import tensorflow as tf
import os



# this time weights form a matrix, not a column vector, one "weight vector" per class.
W = tf.Variable(tf.zeros([4, 3]), name="weights")
# so do the biases, one per class.
b = tf.Variable(tf.zeros([3], name="bias"))



def combine_inputs(X):
    return tf.matmul(X, W) + b


def inference(X):
    return tf.nn.softmax(combine_inputs(X))


def loss(X, Y):
    tf.summary.scalar(tensor = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=combine_inputs(X), labels=Y)), name="loss_summ")
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=combine_inputs(X), labels=Y))



def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([file_name])

    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)

    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)

    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)


def inputs():

    sepal_length, sepal_width, petal_length, petal_width, label =\
        read_csv(100, "iris.data", [[0.0], [0.0], [0.0], [0.0], [""]])

    # convert class names to a 0 based class index.
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.stack([
        tf.equal(label, ["Iris-setosa"]),
        tf.equal(label, ["Iris-versicolor"]),
        tf.equal(label, ["Iris-virginica"])
    ])), 0))

    # Pack all the features that we care about in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.stack([sepal_length, sepal_width, petal_length, petal_width]))

    return features, label_number


def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)



def evaluate(sess, X, Y):

    predicted = tf.cast(tf.arg_max(inference(X), 1), tf.int32)

    print sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32)))



# Launch the graph in a session, setup boilerplate
init = tf.global_variables_initializer()


X, Y = inputs()

total_loss = loss(X, Y)




train_op = train(total_loss)

coord = tf.train.Coordinator()
    
merged_summaries = tf.summary.merge_all()    
writer = tf.summary.FileWriter('./problem3_graph')
writer.add_graph(sess.graph)


with tf.Session() as sess:
    
    sess.run(init)
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)




    # actual training loop
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:
            print "loss: ", sess.run([total_loss])
            writer.add_summary(sess.run(merged_summaries), global_step=step)



    writer.flush()

    # Flushes the summaries to disk and closes the SummaryWriter
    writer.close()  

    evaluate(sess, X, Y)

    coord.request_stop()
    coord.join(threads)
    sess.close()


loss:  [1.0971028]
loss:  [1.0141286]
loss:  [0.94449133]
loss:  [0.9294346]
loss:  [0.87929183]
loss:  [0.83632445]
loss:  [0.79354632]
loss:  [0.78059602]
loss:  [0.73381859]
loss:  [0.7070272]
loss:  [0.67406821]
loss:  [0.66612107]
loss:  [0.65428454]
loss:  [0.66932631]
loss:  [0.64055473]
loss:  [0.62714064]
loss:  [0.59283471]
loss:  [0.59187734]
loss:  [0.59710693]
loss:  [0.6113506]
loss:  [0.66059703]
loss:  [0.56994212]
loss:  [0.59820795]
loss:  [0.55897409]
loss:  [0.58004576]
loss:  [0.52742106]
loss:  [0.54898989]
loss:  [0.53833252]
loss:  [0.51536769]
loss:  [0.51050973]
loss:  [0.51086557]
loss:  [0.49569795]
loss:  [0.47008818]
loss:  [0.47942597]
loss:  [0.50752473]
loss:  [0.47329357]
loss:  [0.51717478]
loss:  [0.49552023]
loss:  [0.52756876]
loss:  [0.52838057]
loss:  [0.53988928]
loss:  [0.53711784]
loss:  [0.4811933]
loss:  [0.46414414]
loss:  [0.47711903]
loss:  [0.45065182]
loss:  [0.4529978]
loss:  [0.42871597]
loss:  [0.45322266]
loss:  [0.43803027]
loss:  